In [1]:
from pathlib import Path

Path('data_app/').parent.absolute()

WindowsPath('c:/Users/Edward TL/OneDrive/Peryton/data_app/reporter/plot')

In [2]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
work_path = 'c:\\Users\\Edward TL\\OneDrive\\Peryton\\data_app'

sys.path.insert(1, work_path)


from helpers import get_config, get_excels

config = get_config(f"{work_path}/extract/dhelos_files.env")
db_path = f"{work_path}\\{config['DHELOS_DB']}"
excels = get_excels(db_path)

In [3]:
import pandas as pd

sells = pd.read_excel(f"{db_path}\\transactions_items_table_api_2024-01-01-2024-05-25.xlsx")
sells.head(5)

,Fecha,Email del cliente,Nombre del cliente,Nombre del Ã­tem,Canal de ventas,Estado del pago,Tipo de servicio,Pedidos totales,Cantidad,Ventas brutas
0,2024-04-23,brendamarinvilla@gmail.com,Brenda Marin,BIRTHDAY CLASSðŸ¥³,channel_type_manual,No pagado,service_type_group,2,2,"18.000,00Â MXN"
1,2024-05-04,brendamarinvilla@gmail.com,Brenda Marin,BIRTHDAY CLASSðŸ¥³,channel_type_manual,Pagado,service_type_group,1,1,"12.600,00Â MXN"
2,2024-05-02,andreamateos95@hotmail.com,Andrea Mateos,FOUNDERS PACK,Web,Pagado,NaN,1,1,"4500,00Â MXN"
3,2024-04-11,athzbal@hotmail.com,Athziry,FOUNDERS PACK,NaN,Pagado,NaN,1,1,"4500,00Â MXN"
4,2024-05-07,marianav@gmx.com,mariana valenzuela,FOUNDERS PACK,com.stripe,Pagado,NaN,1,1,"4500,00Â MXN"


In [4]:
from transform.dhelos_report import dhelos_product_channel_sells

dpcs = dhelos_product_channel_sells(sells, total_row = False, total_col = False)
dpcs[2024][4]

,Nombre del Item,App de Wix,Web,channel_type_manual,com.stripe
0,BIRTHDAY CLASS,0.0,0.0,18000.0,0.0
1,Barre,0.0,0.0,0.0,0.0
2,Clase de Prueba,0.0,0.0,0.0,1800.0
3,FOUNDERS PACK,0.0,0.0,0.0,13500.0
4,Flex,0.0,0.0,0.0,0.0
5,I PACK,0.0,0.0,0.0,750.0
6,IV PACK,0.0,0.0,0.0,5400.0
7,Reformer Clasico,0.0,0.0,0.0,0.0
8,Reformer Contemporaneo,0.0,0.0,0.0,0.0
9,Sculpt,0.0,0.0,0.0,0.0


In [5]:
dpcs[2024][5]

,Nombre del Item,App de Wix,Web,channel_type_manual,com.stripe
0,BIRTHDAY CLASS,0.0,0.0,12600.0,0.0
1,Barre,0.0,0.0,0.0,0.0
2,Clase de Prueba,0.0,600.0,0.0,6100.0
3,FOUNDERS PACK,0.0,9000.0,0.0,9000.0
4,Flex,0.0,0.0,0.0,0.0
5,GUYS DO PILATES TOO,0.0,0.0,0.0,0.0
6,I PACK,0.0,0.0,0.0,1350.0
7,IV PACK,0.0,1680.0,0.0,12500.0
8,Reformer Clasico,0.0,0.0,0.0,0.0
9,Reformer Contemporaneo,0.0,0.0,0.0,0.0


In [6]:
from excel import set_barchart, BAR_TYPES
from openpyxl.chart import Reference, Series
from openpyxl.worksheet.cell_range import CellRange


from dataclasses import dataclass
from objects import ChartData, excel_reference

@dataclass
class Chart:
    _data: ChartData

def bar(Chart: ChartData, bar_type: BAR_TYPES):
    
    chart = set_barchart(bar_type)
    # Basic Data
    chart.title = Chart.titles.chart
    chart.y_axis.title = Chart.titles.y_axis
    chart.x_axis.title = Chart.titles.x_axis

    # Data part
    # All table case
    if Chart.ref.series_values == None:
        chart.add_data(
            Chart.ref.table_values,
            titles_from_data = Chart.ref.labels_at_header
        )
    # Just one column from the table 
    else:
        chart.add_data(
                excel_reference(
                cell_range = CellRange(
                    range_string = Chart.ref.series_values
                    ),
                ws = Chart.data.ws,
                adjust= False
            ),
            titles_from_data = Chart.ref.labels_at_header
        )
        # xvalues = Chart.ref.table_values
        # values = excel_reference(
        #     cell_range = CellRange(
        #         range_string = Chart.ref.series_values
        #         ),
        #     ws = Chart.data.ws,
        #     adjust= False
        # )

        # print("values", values)
        # print("xvalues", xvalues)
        # series = Series(xvalues, values, title_from_data=True)
        # chart.series.append(series)

    # Labels part
    chart.set_categories(
        Chart.ref.first_col
    )

    # Savig the work
    Chart.chart.ws.add_chart(chart, Chart.ref.chart_position)


@dataclass
class Bar(Chart):
    _type: BAR_TYPES = 'normal'

    def __post_init__(self):
        # TO DO
        self.plotly = None

    def to_excel(self):
        return bar(self._data, self._type)
    
    def show(self):
        return self.plotly

In [7]:
import openpyxl

test_file = f"{work_path}\\db\\dhelos\\graph_test.xlsx"
values_sheet = "2024-5"
dpcs[2024][5].to_excel(
    test_file,
    sheet_name = values_sheet,
    index = False
    )
graph_sheet = "Prueba de Grafico"

workbook = openpyxl.load_workbook(test_file)
workbook.create_sheet(graph_sheet)
workbook.save(test_file)

worksheet = workbook[graph_sheet]

In [8]:
dpcs[2024][5].columns

Index(['Nombre del Item', 'App de Wix', 'Web', 'channel_type_manual',
       'com.stripe'],
      dtype='object')

In [9]:
stripe_cols = ['Nombre del Item', 'com.stripe']
wix_cols = ['Nombre del Item', 'Web']
manual = ['Nombre del Item', 'channel_type_manual']

dpcs[2024][5][manual]

,Nombre del Item,channel_type_manual
0,BIRTHDAY CLASS,12600.0
1,Barre,0.0
2,Clase de Prueba,0.0
3,FOUNDERS PACK,0.0
4,Flex,0.0
5,GUYS DO PILATES TOO,0.0
6,I PACK,0.0
7,IV PACK,0.0
8,Reformer Clasico,0.0
9,Reformer Contemporaneo,0.0


In [10]:
from objects import Titles, Palette, ChartReferences, Source, excel_reference

general_data = Source(
    workbook[values_sheet],
    dpcs[2024][5]
)

In [11]:
test_bar1 = Bar(
    _data = ChartData(
        titles = Titles(
            chart = "Ventas de mayo",
            x_axis = "Paquete",
            y_axis = "Monto ($)"
        ),
        data = general_data,
        chart = Source(
            ws = worksheet,
            df = dpcs[2024][5]
        ),
        ref = ChartReferences(
            # chart_sheet = "Report",
            chart_position = "B5",
            labels_at_header = True
        ),
        height = 20,
        width = 30
    )
)
test_bar1.to_excel()

In [12]:
test_bar2 = Bar(
    _data = ChartData(
        titles = Titles(
            chart = "Ventas de Stripe",
            x_axis = "Paquete",
            y_axis = "Monto ($)"
        ),
        data = general_data,
        chart = Source(
            ws = worksheet,
            df = dpcs[2024][5][stripe_cols]
        ),
        ref = ChartReferences(
            # chart_sheet = "Report",
            chart_position = "L5",
            labels_at_header = True,
            series_values = "E1:E15"
            ),
        
        height = 20,
        width = 30
    )
)
test_bar2.to_excel()

In [13]:
test_bar3 = Bar(
    _data = ChartData(
        titles = Titles(
            chart = "Ventas de Manual",
            x_axis = "Paquete",
            y_axis = "Monto ($)"
        ),
        data = general_data,
        chart = Source(
            ws = worksheet,
            df = dpcs[2024][5][manual]
        ),
        ref = ChartReferences(
            # chart_sheet = "Report",
            chart_position = "B20",
            labels_at_header = True,
            series_values = "D1:D15"
        ),
        
        height = 20,
        width = 30
    )
)
test_bar3.to_excel()

In [14]:
test_bar4 = Bar(
    _data = ChartData(
        titles = Titles(
            chart = "Ventas de Wix",
            x_axis = "Paquete",
            y_axis = "Monto ($)"
        ),
        data = general_data,
        chart = Source(
            ws = worksheet,
            df = dpcs[2024][5][wix_cols]
        ),
        ref = ChartReferences(
            # chart_sheet = "Report",
            chart_position = "L20",
            series_values = "C1:C15",
            labels_at_header = True
        ),
        
        height = 20,
        width = 30
    )
)
test_bar4.to_excel()

In [15]:
workbook.save(test_file)
workbook.close()